In [1]:
library(ArchR)
library(org.Ss.eg.db)
library(SuscrofaTxdb.11.108.july)
library(Repitools)


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__                     ||      \\\
        ,--' ,----`-,__ ___/'  --,-`-===================##========>
       \               '        ##_______ _____ ,--,__,=##,__   ///
        ,    __==    ___,-,__,--'#'  ==='      `-'    | ##,-/
        -,____,---'       \\####\\________________,--\\_##,/
           ___      .______        ______  __    __  .______      
          /   \     |   _ 

In [2]:
options(repr.plot.width = 18, repr.plot.height = 17, repr.plot.pointsize = 24)

In [3]:
addArchRThreads(5)

Setting default number of Parallel threads to 5.



In [4]:
load(file = "/home/adufour/work/rds_storage/omics/archr_all_v7_stemcells.RData")

In [5]:
peak_grange <- archrproj_sub@peakSet

In [6]:
peak_grange$peakName <- peak_grange %>% {paste0(seqnames(.), "_", start(.), "_", end(.))}

In [7]:
names(peak_grange) <- NULL

In [8]:
peak_DF <- annoGR2DF(peak_grange)

In [10]:
markersPeaks <- getMarkerFeatures(
    ArchRProj = archrproj_sub, 
    useMatrix = "PeakMatrix", 
    groupBy = "Clusters",
    bias = c("TSSEnrichment", "log10(nFrags)"),
    testMethod = "wilcoxon"
)

ArchR logging to : ArchRLogs/ArchR-getMarkerFeatures-25cb5574df97de-Date-2024-07-04_Time-17-09-02.log
If there is an issue, please report to github with logFile!

MatrixClass = Sparse.Integer.Matrix

2024-07-04 17:09:03 : Matching Known Biases, 0.009 mins elapsed.

Warning message:
“le package ‘nabor’ a été compilé avec la version R 4.2.3”
###########
2024-07-04 17:36:48 : Completed Pairwise Tests, 27.772 mins elapsed.
###########

ArchR logging successful to : ArchRLogs/ArchR-getMarkerFeatures-25cb5574df97de-Date-2024-07-04_Time-17-09-02.log



In [11]:
markerPeaksList <- getMarkers(markersPeaks, cutOff = "FDR <= 0.01 & Log2FC >= 1.25")

In [13]:
markerPeaksList$C2$peakName <- paste0(markerPeaksList$C2$seqnames, "_", markerPeaksList$C2$start, "_", markerPeaksList$C2$end)

In [15]:
write.csv(merge(markerPeaksList$C2, peak_DF, by = "peakName"), "/home/adufour/work/table/c2_stemcells_peak_marker_with_gene.csv", row.names=FALSE)

In [13]:
peakDF <- metadata(archrproj_sub@peakSet)$PeakCallSummary

In [14]:
getP2G_GR <- function(proj, corrCutoff=NULL, varCutoffATAC=0.25, varCutoffRNA=0.25, filtNA=TRUE){
  # Function to get peaks and genes involved in peak to gene links
  # (See: https://github.com/GreenleafLab/ArchR/issues/368)
  ############################################################
  # proj: ArchR project that alreayd has Peak2GeneLinks
  # corrCutoff: minimum numeric peak-to-gene correlation to return
  # varCutoffATAC: minimum variance quantile of the ATAC peak accessibility when selecting links
  # varCutoffRNA: minimum variance quantile of the RNA gene expression when selecting links
  p2gDF <- metadata(proj@peakSet)$Peak2GeneLinks
  p2gDF$symbol <- mcols(metadata(p2gDF)$geneSet)$name[p2gDF$idxRNA] %>% as.character()
  p2gDF$peakName <- (metadata(p2gDF)$peakSet %>% {paste0(seqnames(.), "_", start(.), "_", end(.))})[p2gDF$idxATAC]
  # Remove peaks with 'NA' correlation values
  if(filtNA){
    p2gDF <- p2gDF[!is.na(p2gDF$Correlation),]
  }
  if(!is.null(corrCutoff)){
    p2gDF <- p2gDF[(p2gDF$Correlation > corrCutoff),]
  }
  # Filter by variance quantile
  p2gDF <- p2gDF[which(p2gDF$VarQATAC > varCutoffATAC & p2gDF$VarQRNA > varCutoffRNA),]
  # The genomic range contains just the peak ranges:
  p2gGR <- metadata(p2gDF)$peakSet[p2gDF$idxATAC]
  mcols(p2gGR) <- p2gDF
  p2gGR
}

In [15]:
peak2gene_list <- list()

In [16]:
p2gGR <- getP2G_GR(archrproj_sub, corrCutoff=NULL, varCutoffATAC=-Inf, varCutoffRNA=-Inf, filtNA=FALSE)

In [17]:
p2gGR_DF <- annoGR2DF(p2gGR)

In [18]:
df1 <- merge(markerPeaksList$C2, peak_DF, by = "peakName")

ERROR: Error in fix.by(by.x, x): 'by' doit spécifier une colonne unique correcte


In [ ]:
df2 <- merge(df1, p2gGR_DF, by = "peakName")

In [23]:
write.csv(df2, "/home/adufour/work/table/c2_stemcells_peak_marker_with_gene_and_p2g.csv", row.names=FALSE)

In [19]:
heatmapPeaks <- ArchR::plotMarkerHeatmap(
  seMarker = markersPeaks,
  cutOff = "FDR <= 0.1 & Log2FC >= 0.5",
  transpose = FALSE,
  returnMatrix = TRUE
)

ArchR logging to : ArchRLogs/ArchR-plotMarkerHeatmap-25cb557de52231-Date-2024-07-04_Time-17-40-52.log
If there is an issue, please report to github with logFile!

Identified 28621 markers!



 [1] "1:249812-250312"       "1:251770-252270"       "1:397879-398379"      
 [4] "1:452471-452971"       "1:505852-506352"       "1:1098350-1098850"    
 [7] "1:1274246-1274746"     "1:1288161-1288661"     "1:1334232-1334732"    
[10] "1:1477528-1478028"     "1:1550552-1551052"     "1:1723878-1724378"    
[13] "1:1725716-1726216"     "1:1729878-1730378"     "1:1749487-1749987"    
[16] "1:90413-90913"         "1:242325-242825"       "1:407452-407952"      
[19] "1:555626-556126"       "1:839051-839551"       "1:1875684-1876184"    
[22] "1:1942209-1942709"     "1:1942929-1943429"     "1:1982525-1983025"    
[25] "1:2211233-2211733"     "1:2337818-2338318"     "1:2361409-2361909"    
[28] "1:2628371-2628871"     "1:2720478-2720978"     "1:2829161-2829661"    
[31] "1:2004035-2004535"     "1:7459180-7459680"     "1:10215378-10215878"  
[34] "1:11950770-11951270"   "1:12374771-12375271"   "1:17410623-17411123"  
[37] "1:17513875-17514375"   "1:34758842-34759342"   "1:52987383-52987883"  

ArchR logging successful to : ArchRLogs/ArchR-plotMarkerHeatmap-25cb557de52231-Date-2024-07-04_Time-17-40-52.log



In [20]:
heatmapPeaks <- as.data.frame(heatmapPeaks)
heatmapPeaks$peaks <- rownames(heatmapPeaks)

In [21]:
peak_grange <- archrproj_sub@peakSet

In [22]:
peak_grange$peakName <- peak_grange %>% {paste0(seqnames(.), ":", start(.), "-", end(.))}

In [23]:
names(peak_grange) <- NULL

In [24]:
peak_DF <- annoGR2DF(peak_grange)

In [25]:
peak_DF <- peak_DF[peak_DF$peakType == "Promoter",]

In [26]:
heatmapPeaks <- heatmapPeaks[heatmapPeaks$peaks %in% peak_DF$peakName,]

In [27]:
colnames(heatmapPeaks) <- c("C2", "C4", "C6", "peakName")

In [28]:
write.csv(merge(heatmapPeaks, peak_DF, by = "peakName"), "/home/adufour/work/table/heatmap_stemcells_peak.csv", row.names=FALSE)

In [29]:
heatmapPeaks <- heatmapPeaks[,c("C6","C2","C4")]

In [30]:
library(svglite)
library(ComplexHeatmap)
library(circlize)
library(RColorBrewer)

Warning message:
“le package ‘svglite’ a été compilé avec la version R 4.2.3”
ComplexHeatmap version 2.14.0
Bioconductor page: http://bioconductor.org/packages/ComplexHeatmap/
Github page: https://github.com/jokergoo/ComplexHeatmap
Documentation: http://jokergoo.github.io/ComplexHeatmap-reference

If you use it in published research, please cite either one:
- Gu, Z. Complex Heatmap Visualization. iMeta 2022.
- Gu, Z. Complex heatmaps reveal patterns and correlations in multidimensional 
    genomic data. Bioinformatics 2016.


The new InteractiveComplexHeatmap package can directly export static 
complex heatmaps into an interactive Shiny app with zero effort. Have a try!

This message can be suppressed by:
  suppressPackageStartupMessages(library(ComplexHeatmap))


Warning message:
“le package ‘circlize’ a été compilé avec la version R 4.2.3”
circlize version 0.4.15
CRAN page: https://cran.r-project.org/package=circlize
Github page: https://github.com/jokergoo/circlize
Documentation: h

In [31]:
col_fun <- colorRamp2(seq(min(heatmapPeaks), max(heatmapPeaks), length = 100), colorRampPalette(rev(brewer.pal(n = 4, name = "RdYlBu")))(100))
lgd <- list(title = "Column Z-Scores", legend_height = unit(6, "cm"), grid_width = unit(1, "cm"), labels_gp = gpar(fontsize = 20), title_gp = gpar(fontsize = 18), direction = "horizontal")
svglite('/home/adufour/work/notebook/plots/omics/heatmap_peaks_stemcells.svg', width = 18, height = 5)
hm <- draw(ComplexHeatmap::Heatmap(t(heatmapPeaks), col = col_fun, show_column_names = FALSE, cluster_rows = FALSE ,
                                   row_names_gp = grid::gpar(fontsize=20), heatmap_legend_param = lgd), heatmap_legend_side = "bottom",
                                   padding = unit(c(2, 8, 2, 2), "mm"))
dev.off()

png 
  2

# export DAG whole

In [11]:
scriptPath <- "/home/adufour/work/scScalpChromatin"
source(paste0(scriptPath, "/misc_helpers.R"))
source(paste0(scriptPath, "/matrix_helpers.R"))
source(paste0(scriptPath, "/plotting_config.R"))
source(paste0(scriptPath, "/archr_helpers.R"))

Warning message:
“le package ‘dplyr’ a été compilé avec la version R 4.2.3”
Warning message:
“le package ‘tidyr’ a été compilé avec la version R 4.2.3”
Warning message:
“le package ‘RcppAlgos’ a été compilé avec la version R 4.2.3”
Warning message:
“le package ‘ggrastr’ a été compilé avec la version R 4.2.3”


In [12]:
corrCutoff <- 0.5
p2gGR <- getP2G_GR(archrproj_sub, corrCutoff=corrCutoff)

In [13]:
p2geneDF <- Repitools::annoGR2DF(p2gGR)

In [14]:
markerPeaksList <- getMarkers(markersPeaks, cutOff = "FDR <= 0.1 & abs(Log2FC) >= 1")

In [16]:
markerPeaksList$C2$peakName <- paste0(markerPeaksList$C2$seqnames, "_", markerPeaksList$C2$start, "_", markerPeaksList$C2$end)
markerPeaksList$C4$peakName <- paste0(markerPeaksList$C4$seqnames, "_", markerPeaksList$C4$start, "_", markerPeaksList$C4$end)
markerPeaksList$C6$peakName <- paste0(markerPeaksList$C6$seqnames, "_", markerPeaksList$C6$start, "_", markerPeaksList$C6$end)

In [19]:
peak_grange <- archrproj_sub@peakSet

In [20]:
peak_grange$peakName <- peak_grange %>% {paste0(seqnames(.), "_", start(.), "_", end(.))}

In [21]:
names(peak_grange) <- NULL

In [22]:
peak_DF <- Repitools::annoGR2DF(peak_grange)

In [23]:
merge_C2 <- merge(markerPeaksList$C2,p2geneDF,by="peakName", all.x=TRUE)
merge_C4 <- merge(markerPeaksList$C4,p2geneDF,by="peakName", all.x=TRUE)
merge_C6 <- merge(markerPeaksList$C6,p2geneDF,by="peakName", all.x=TRUE)

In [24]:
write.csv(merge(merge_C2,peak_DF,by="peakName"), "/home/adufour/work/table/omics_whole_dag_c2.csv", row.names=TRUE)
write.csv(merge(merge_C4,peak_DF,by="peakName"), "/home/adufour/work/table/omics_whole_dag_c4.csv", row.names=TRUE)
write.csv(merge(merge_C6,peak_DF,by="peakName"), "/home/adufour/work/table/omics_whole_dag_c6.csv", row.names=TRUE)

In [10]:
write.csv(peak_DF, "/home/adufour/work/table/peak_DF_stemcells.csv", row.names=FALSE)